In [1]:
import numpy as np
import math

In [2]:
#electrode matrices
electrode_voltages = np.zeros((8,8))
electrode_positions = np.zeros((8,8,2))

#RGC matrices
on_parasol_responses = np.zeros((2*28,16)) #28 rows in hexagonal grid means 28 rows on a straight, but 56 rows staggered
on_parasol_positions = np.zeros((2*28,16,2)) #32 cols in hex grid means 32 staggered cols = 16 straight cols

#Bipolar matrices 512x512 area with 2 micron pitch, so 1 neuron every 2 microns, so 256x256 array

on_parasol_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
on_parasol_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

In [3]:
#set positions

#electrodes:
#512 micron patch of retina and 70 micron electrode pitch
#odd rows start at x=0 and go to 0+70*7 = 490
#even rows start at x=512 and go down to 512-70*7 = 22 (x=22 to x=512 by 70)
#y values are evenly spread from y=16 to about y=506 (by 70)

y_positions = np.arange(16,507,70)

i = 0
while(i<np.shape(electrode_positions)[0]): #go through all rows, use np.arrange to evenly space electrodes
    #set y position for this row
    electrode_positions[i,:,1] = np.ones(8)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        electrode_positions[i,:,0] = np.arange(22, 513, 70)
    else:
        electrode_positions[i,:,0] = np.arange(0, 491, 70)
    i+=1

    

#on parasol rgc's:
#512 micron patch of retina, 28x32 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#28.5*height of hexagon = 512, h ~ 17.96 microns, y value increases by 1/2 h each row
#16*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/24.75 ~ 20.7 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/28.5
w=512/24.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_positions[i,:,1] = np.ones(16)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1

    
    
#on parasol bipolar cells
#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


In [4]:
# get electrode voltages from image

# random data:

electrode_voltages = np.random.rand(8,8) * 8 #lets assume up to 8 mV for now
electrode_voltages

array([[0.40980033, 1.79463392, 7.71729727, 7.01105071, 7.75975869,
        5.91453651, 0.50738802, 0.63361294],
       [4.84133406, 6.99361956, 1.78043673, 7.6026307 , 2.15962022,
        0.1077956 , 4.23823854, 5.41507448],
       [3.44464661, 5.5816058 , 0.87064511, 1.77072431, 1.48812819,
        1.97571153, 3.15016986, 6.34427894],
       [1.63247307, 6.02360818, 4.51004107, 0.05721628, 0.77967395,
        6.17741694, 7.17905447, 2.84869229],
       [7.0509566 , 6.13514501, 7.13464656, 0.94236924, 0.58059058,
        2.23686552, 3.06213599, 5.66522846],
       [2.13926254, 3.72150821, 4.52112178, 3.62553657, 5.83062239,
        4.08040936, 4.31737971, 2.86693985],
       [7.30270347, 1.76473224, 2.44886221, 1.3460623 , 6.8500462 ,
        0.93464305, 3.72801712, 7.21285865],
       [3.23264533, 4.51191045, 3.54619928, 2.08371104, 4.29344201,
        1.50824188, 6.79347485, 2.40619632]])

In [5]:
# get response of bipolar cells from electrodes


#spacial response

#loop through all electrodes, find "bounding box" (within ~35 microns of electrode in each direction),
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation

i=0
j=0
while(i<np.shape(electrode_positions)[0]):
    while(j<np.shape(electrode_positions)[1]):
        x_elec = electrode_positions[i,j,0]
        y_elec = electrode_positions[i,j,1]
        x1 = x_elec - 35
        x2 = x_elec + 35
        y1 = y_elec - 35
        y2 = y_elec + 35
        
        voltage = electrode_voltages[i,j]
        
        #go through all neurons and simulate responses
        
        #on parasol bipolar simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_bipolar_positions)[0]):
            while(b<np.shape(on_parasol_bipolar_positions)[1]):
                x=on_parasol_bipolar_positions[a,b,0]
                y=on_parasol_bipolar_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    on_parasol_bipolar_responses[a,b] += G*voltage
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        
        #other bipolar simulation:
        a=0
        b=0
        #...
        
        j+=1
    j=0
    i+=1


#temporal response


#Since golden paper didn't give any information on the temporal filter (and we are using slightly different
#temporal modeling with single frame of picture shown then nothing for the rest of the 0.4 s), we will use
#exponential decay to model the temporal behavior with the initial value starting at the spacial response
#and the decay value to be 50 (0.02s half life of neuron response strength, close to curve of golden
#temporal response)

#response = init_response * e^(-50*t)

initial_on_parasol_bipolar_responses = np.copy(on_parasol_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(on_parasol_bipolar_positions)[0]):
        while(j<np.shape(on_parasol_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_on_parasol_bipolar_responses[i,j]
            
            response = init_response * math.e**(-50*t)
            
            on_parasol_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001

In [6]:
#simulate rgc responses to bipolars

#loop through all bipolars, find "bounding box" (within ~30 microns of electrode in each direction),
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation


#on parasol rgc:

i=0
j=0
while(i<np.shape(on_parasol_bipolar_positions)[0]):
    while(j<np.shape(on_parasol_bipolar_positions)[1]):
        x_elec = on_parasol_bipolar_positions[i,j,0]
        y_elec = on_parasol_bipolar_positions[i,j,1]
        x1 = x_elec - 30
        x2 = x_elec + 30
        y1 = y_elec - 30
        y2 = y_elec + 30
        
        response = on_parasol_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #on parasol rgc simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_positions)[0]):
            while(b<np.shape(on_parasol_positions)[1]):
                x=on_parasol_positions[a,b,0]
                y=on_parasol_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.375
                    sig_cent = 10
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    on_parasol_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1



In [8]:
#convert response to firing rate


#go through all rgc responses and convert response to firing rate

#-----------------------------------------------------------------------------------------------
#important: adjust the exponential function later once voltages are adjusted

#we'll use an exponential function to get spike rate from rgc response (golden got this from pillow et al 2008)
#large rgc response is 0.1, so exponential goes from e^0=1 to e^0.1=1.1
#6 Hz seems to be a large neuron firing rate for an RGC (from other studies), so we'll use this as max_rate
#exponential function: firing_rate = max_rate*10*(e^(response)-1)
#subtract 1 from exponential so we have values starting from 0, multiply by 10 to get our max firing rate
#to be about equal to our inputted max firing rate

max_rate = 6 #6Hz seems to be a very large RGC firing rate

#hopefully the learned reconstruction matrix will be able to compensate for any differences we've made in
#our equations (as long as we use the same equations here and in learning)

i=0
j=0
while(i<np.shape(on_parasol_positions)[0]):
    while(j<np.shape(on_parasol_positions)[1]):
        response = on_parasol_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        on_parasol_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1
